# VF 02 - RAG

References:
 - [Langchain Ollama Integration](https://python.langchain.com/docs/integrations/llms/ollama/)
 - [Ollama Server Hardware Spec](https://everymac.com/systems/apple/mac_mini/specs/mac-mini-m4-10-core-cpu-10-core-gpu-2024-specs.html)




In [10]:
%pip install -q langchain langchain-community langchain-ollama langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [27]:
import os
from uuid import uuid4
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA

In [25]:
# Globals
# from langchain import globals
#globals.set_debug(True)

dataset_directory = "/mnt/network-data/storage/projects/vf/data"

ollama_api_endpoint = "http://moon.btech.software:11434"

embedding_model = "phi4"
qa_model = "phi4"

kb_dir=f"{dataset_directory}/DataStock"


In [26]:
embeddings = OllamaEmbeddings(model=embedding_model, base_url=ollama_api_endpoint)

vector_store = Chroma(
    collection_name="datastock_collection",
    embedding_function=embeddings,
    persist_directory="./tmp/datastock_collection.bin",
)

for root, dirs, files in os.walk(kb_dir):
    for file in files:
        if file.lower().endswith("_processed.txt"):
            file_path = os.path.join(root, file)
            print(f"Processing {file_path}")
            text_loader = TextLoader(file_path)
            docs = text_loader.load_and_split()
            uuids = [str(uuid4()) for _ in range(len(docs))]
            vector_store.add_documents(documents=docs, ids=uuids)



Processing /mnt/network-data/storage/projects/vf/data/DataStock/Guia RENAVE_atualizado-page-1_processed.txt
Processing /mnt/network-data/storage/projects/vf/data/DataStock/Guia RENAVE_atualizado-page-2_processed.txt
Processing /mnt/network-data/storage/projects/vf/data/DataStock/Guia RENAVE_atualizado-page-3_processed.txt
Processing /mnt/network-data/storage/projects/vf/data/DataStock/Guia RENAVE_atualizado-page-4_processed.txt
Processing /mnt/network-data/storage/projects/vf/data/DataStock/Guia RENAVE_atualizado-page-5_processed.txt
Processing /mnt/network-data/storage/projects/vf/data/DataStock/Guia RENAVE_atualizado-page-6_processed.txt
Processing /mnt/network-data/storage/projects/vf/data/DataStock/Guia RENAVE_atualizado-page-7_processed.txt
Processing /mnt/network-data/storage/projects/vf/data/DataStock/Guia RENAVE_atualizado-page-8_processed.txt
Processing /mnt/network-data/storage/projects/vf/data/DataStock/Guia RENAVE_atualizado-page-9_processed.txt
Processing /mnt/network-data

In [28]:
# initialize chain
llm = OllamaLLM(
    model=qa_model,
    base_url=ollama_api_endpoint,
    temperature=0.5,
    num_predict=1024
)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # map_reduce, refine, map_rerank
    retriever=vector_store.as_retriever(),
    verbose=True
)

In [29]:
query = "Responda em pt_BR: Como o RENAVE pode auxiliar os clientes?"
response = qa_chain.run(query)
print(response)

/tmp/ipykernel_960876/3965892567.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)




> Entering new RetrievalQA chain...

> Finished chain.
O RENAVE, que é uma agência brasileira responsável pelo transporte rodoviário e circulação de veículos, pode auxiliar seus clientes principalmente através dos seguintes serviços:

1. **Transferência de Veículos**: O RENAVE facilita o processo de transferência de veículos entre estabelecimentos, garantindo que todas as etapas sejam cumpridas conforme os regulamentos vigentes. Isso é essencial para empresas envolvidas na comercialização de veículos.

2. **Comercialização de Veículos**: Ao fornecer informações claras e precisas sobre o processo de transferência, o RENAVE ajuda as empresas a otimizar suas operações de venda e compra de veículos, reduzindo custos e melhorando a satisfação do cliente.

3. **Autorização para Transferência entre Estabelecimentos**: O RENAVE emite autorizações necessárias para que os estabelecimentos possam realizar transferências legais e seguras de veículos, garantindo conformidade com as normas regulat